### Setting everything up

In [1]:
#importing the read_sbml_model function from cobra
from cobra.io import read_sbml_model
#make sure model name is up to date!
model = read_sbml_model('last_version.xml') #assigning our model to the variable model

No objective coefficients in model. Unclear what should be optimized


In [2]:
model.optimize().objective_value

0.0

In [3]:
# no objective function is set in the published model iCW773, thus optimization eturns 0
#setting the objective to the biomass reaction of Corynebacterium glutamicum
#find the biomass function:
model.reactions.query("biomass", "id")

[<Reaction CG_biomass_cgl_ATCC13032 at 0x22340277c10>,
 <Reaction EX_biomass at 0x223405edb50>]

In [4]:
#CG_biomass_cgl_ATCC13032 is the id of the biomass reaction 
#setting it as objective
model.objective = model.reactions.CG_biomass_cgl_ATCC13032

#optimze the model for biomass production: 
model.optimize().objective_value

0.4289296621348934

In [5]:
# now we get a value and can start using the model.

## Co-factor swap 

In [6]:
#cameo provides a package to perform co-factor swapping
from cameo.strain_design.heuristic.evolutionary_based import CofactorSwapOptimization

In [7]:
#computing maximum growth rate
with model:
    max_growth = model.slim_optimize()
    print("Corynabcterium glutamicum has a maximum growth rate of: ",max_growth)


Corynabcterium glutamicum has a maximum growth rate of:  0.4289296621348934


In [8]:
#searching for the id of psilocybin
model.metabolites.query("psi", "id")

[<Metabolite psii_c at 0x2234012b280>, <Metabolite psi_c at 0x2234012b2b0>]

In [9]:
model.metabolites.psi_c #calling the metabolite information of psilocybin

Metabolite identifier,psi_c
Name,Psilocybin
Memory address,0x02234012b2b0
Formula,C12H17N2O4P
Compartment,c
In 4 reaction(s),"psiM2, pho, psiK2, DM_psi_c"


In [10]:
model.reactions.psiM2 # reaction producing psilocybin from Baecystin

Reaction identifier,psiM2
Name,
Memory address,0x022340705d90
Stoichiometry,amet_c + baecyst_c --> ahcys_c + psi_c S-AdenosyL_L_methionine + Baeocystin --> S-AdenosyL_L_homocysteine + Psilocybin
GPR,
Lower bound,0.0
Upper bound,1000.0


In [11]:
model.reactions.psiK2 # reaction, which is producing psilocybin after spontaneous degradation 

Reaction identifier,psiK2
Name,
Memory address,0x022340705fd0
Stoichiometry,atp_c + psii_c --> adp_c + psi_c ATP + psilocin --> ADP + Psilocybin
GPR,
Lower bound,0.0
Upper bound,1000.0


In [58]:
model.reactions.DM_psi_c

Reaction identifier,DM_psi_c
Name,Psilocybin demand
Memory address,0x022340705ee0
Stoichiometry,psi_c --> Psilocybin -->
GPR,
Lower bound,0.0
Upper bound,1000.0


In [57]:
with model:
    model.objective = model.reactions.DM_psi_c
    psilocybin_max_production = model.slim_optimize()
    print("Corynebacterium has a max. production rate of: ", psilocybin_max_production)

Corynebacterium has a max. production rate of:  nan


In [13]:
# now we'll do the cofactor swapping:

In [18]:
from cameo.strain_design.heuristic.evolutionary.objective_functions import product_yield

In [19]:
CofactorSwapOptimization?

In [23]:
model.metabolites.query("glc", "id")

[<Metabolite 5dh4dglc_c at 0x2233e0f11f0>,
 <Metabolite acglc_D_c at 0x2233e0f1700>,
 <Metabolite adpglc_c at 0x2233e0f1be0>,
 <Metabolite glc_D_c at 0x223400be070>,
 <Metabolite glc_D_e at 0x223400be160>,
 <Metabolite glcn_c at 0x223400be190>,
 <Metabolite glcn_e at 0x223400be0a0>,
 <Metabolite glcr_c at 0x223400cd130>,
 <Metabolite glcur_c at 0x223400cd0d0>,
 <Metabolite glcur_e at 0x223400cd100>,
 <Metabolite glcur1p_c at 0x223400cd1c0>,
 <Metabolite glcur1p_e at 0x223400cd1f0>,
 <Metabolite udpglcur_c at 0x2234011bb50>,
 <Metabolite udpglcur_e at 0x2234011bb80>]

In [25]:
model.metabolites.glc_D_e

Metabolite identifier,glc_D_e
Name,D_Glucose
Memory address,0x0223400be160
Formula,C6H12O6
Compartment,c
In 3 reaction(s),"GLCptspp, EX_glc_e, GLCabcpp"


In [26]:
model.objective = model.reactions.DM_psi_c
model.reactions.CG_biomass_cgl_ATCC13032.lower_bound = 0.1
py = product_yield(model.reactions.DM_psi_c, model.reactions.EX_glc_e)
swap_optimization = CofactorSwapOptimization(model=model, objective_function=py)
swap_optimization.run(max_evaluations=2000, max_size=2) # I copied this fromn the example, but don't understand why these parameters are selected

Starting optimization at Thu, 18 Nov 2021 15:48:12


HBox()

Finished after 00:00:32


,index,targets,fitness


In [27]:
# maybe our co-factors are named differently cofactor_id_swaps=(['NADP', 'NADPH'], ['NAD', 'NADH'])

In [31]:
model.metabolites.query("nadph", "id")

[<Metabolite nadph_c at 0x223400ed610>]

In [32]:
model.metabolites.query("nadp", "id")

[<Metabolite nadp_c at 0x223400ed5e0>, <Metabolite nadph_c at 0x223400ed610>]

In [33]:
model.metabolites.query("nadh", "id")

[<Metabolite nadh_c at 0x223400ed5b0>]

In [34]:
model.metabolites.query("nad", "id")

[<Metabolite dnad_c at 0x223400affa0>,
 <Metabolite nad_c at 0x223400ed580>,
 <Metabolite nadh_c at 0x223400ed5b0>,
 <Metabolite nadp_c at 0x223400ed5e0>,
 <Metabolite nadph_c at 0x223400ed610>]

In [ ]:
#cofactor_id_swaps=(['nadp_c', 'nadph_c'], ['nad_c', 'nadh_c'])

In [53]:
model.metabolites.nad_c

Metabolite identifier,nad_c
Name,Nicotinamide adenine dinucleotide
Memory address,0x0223400ed580
Formula,C21H26N7O14P2
Compartment,c
In 59 reaction(s),"NADH9, MECDPDH2, MI2DH, PDH, LDH_D, AKGDH, PGCD, M1PD, EAR180x, MDH, ALCD19, NADPPPS, DMPPS, HISTD, SOL_POOL, ALDD19x, EAR80x, NADH10, ACALD, EAR181x, ALCD2x, EAR100x, LDH, IPMD, NADH17pp,..."


In [35]:
model.objective = model.reactions.DM_psi_c
model.reactions.CG_biomass_cgl_ATCC13032.lower_bound = 0.1
py = product_yield(model.reactions.DM_psi_c, model.reactions.EX_glc_e)
swap_optimization = CofactorSwapOptimization(model=model, objective_function=py,cofactor_id_swaps=(['nadp_c', 'nadph_c'], ['nad_c', 'nadh_c']))
swap_optimization.run(max_evaluations=2000, max_size=2) # I copied this fromn the example, but don't understand why these parameters are selected

Starting optimization at Thu, 18 Nov 2021 16:10:47


HBox()

Finished after 00:00:34


,index,targets,fitness


In [38]:
model.reactions.EX_glc_e

Reaction identifier,EX_glc_e
Name,D_Glucose exchange
Memory address,0x0223403dcac0
Stoichiometry,glc_D_e <-- D_Glucose <--
GPR,
Lower bound,-4.67
Upper bound,-4.09906


In [39]:
model.medium

{'EX_ca2_e': 0.0144877134286247,
 'EX_cobalt2_e': 7.00338234344926e-05,
 'EX_cu2_e': 0.00197271722237563,
 'EX_fe2_e': 0.16239125,
 'EX_fe3_e': 0.0217300637039616,
 'EX_glc_e': 4.67,
 'EX_k_e': 0.543285250115532,
 'EX_mg2_e': 0.0241459221505149,
 'EX_mn2_e': 0.00192443552016793,
 'EX_nh4_e': 11.9121416666662,
 'EX_ni2_e': 0.00089824250292982,
 'EX_o2_e': 16.5067783333334,
 'EX_pi_e': 0.918000000000888,
 'EX_so4_e': 4.01673156249995,
 'EX_zn2_e': 0.000948926279874219,
 'EX_BIOTIN': 1.01925397503027e-06,
 'EX_cl': 0.0144877134286247}

In [43]:
model.metabolites.query("psi", "id")

[<Metabolite psii_c at 0x2234012b280>, <Metabolite psi_c at 0x2234012b2b0>]

In [44]:
model.metabolites.psi_c

Metabolite identifier,psi_c
Name,Psilocybin
Memory address,0x02234012b2b0
Formula,C12H17N2O4P
Compartment,c
In 4 reaction(s),"psiM2, pho, psiK2, DM_psi_c"


In [45]:
model.objective = model.reactions.DM_psi_c
model.reactions.CG_biomass_cgl_ATCC13032.lower_bound = 0.1
py = product_yield(model.reactions.DM_psi_c, model.reactions.EX_glc_e)
swap_optimization = CofactorSwapOptimization(model=model, objective_function=py,cofactor_id_swaps=(['nadp_c', 'nadph_c'], ['nad_c', 'nadh_c']))
swap_optimization.run(max_evaluations=2000, max_size=2) # I copied this fromn the example, but don't understand why these parameters are selected

Starting optimization at Thu, 18 Nov 2021 16:21:05


HBox()

Finished after 00:00:30


,index,targets,fitness


In [46]:
py

In [50]:
model.reactions.EX_glc_e.lower_bound = 0
model.reactions.EX_glc_e.upper_bound = 10

model.solver = "glpk" # found this somewhere

In [51]:
model.objective = model.reactions.DM_psi_c
model.reactions.CG_biomass_cgl_ATCC13032.lower_bound = 0.1
py = product_yield(model.reactions.DM_psi_c, model.reactions.EX_glc_e)
swap_optimization = CofactorSwapOptimization(model=model, objective_function=py,cofactor_id_swaps=(['nadp_c', 'nadph_c'], ['nad_c', 'nadh_c']))
swap_optimization.run(max_evaluations=2000, max_size=2) # I copied this fromn the example, but don't understand why these parameters are selected

Starting optimization at Thu, 18 Nov 2021 16:26:54


HBox()

Finished after 00:00:09


,index,targets,fitness


In [52]:
# this is also not giving results

In [55]:
py = product_yield(model.reactions.DM_psi_c, model.reactions.EX_glc_e)
swap_opt = CofactorSwapOptimization(model=model, objective_function=py, plot=True, cofactor_id_swaps=(['nadp_c', 'nadph_c'], ['nad_c', 'nadh_c']))
%time
result = swap_opt.run(max_size=5)

Wall time: 0 ns
Starting optimization at Thu, 18 Nov 2021 16:32:04


HBox()

Finished after 00:00:04


In [56]:
result

,index,targets,fitness


In [59]:
print(model.objective.expression)
print(model.objective.direction)

1.0*DM_psi_c - 1.0*DM_psi_c_reverse_8fa95
max


In [61]:
model.reactions.get_by_id("EX_o2_e")

Reaction identifier,EX_o2_e
Name,O2 exchange
Memory address,0x0223403efd60
Stoichiometry,o2_e <-- O2 <--
GPR,
Lower bound,-16.5067783333334
Upper bound,-1.44076799999993
